In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp pipeline

In [ ]:
# hide

# TODO: see if there are nicer ways of importing from other notebooks
import sys
sys.path.insert(1, './kaggle-m5-nbdev') 
from core import test_eq, test_err, configure_logging
log = configure_logging('./tmp', 'test_log', con_log_lvl='DEBUG')

Logging already initialized


In [ ]:
#export

from core import read_series_sample, melt_sales_series, extract_day_ids, join_w_calendar, join_w_prices
from core import to_parquet, get_submission_template_melt
import os

def prepare_data_on_disk(log, n_sample_series, processed):
    expected_path = f'{processed}/sales_series_melt.parquet'
    if os.path.exists(expected_path) and not force_data_prep:
        log.info(f'Found parquet file ({expected_path})- skipping the prep')
        return

    log.info(f'Not found parquet file ({expected_path}) - preparing the data')

    sales_series = read_series_sample(log, n_sample_series)
    sales_series = melt_sales_series(sales_series)
    sales_series = extract_day_ids(sales_series)
    sales_series = join_w_calendar(sales_series)
    sales_series = join_w_prices(sales_series).persist()
    to_parquet(sales_series, 'sales_series_melt.parquet', processed, log)

In [ ]:
prepare_data_on_disk(log, n_sample_series=10, processed='./tmp')

 INFO [2020-07-13 20:19:09] root: Not found parquet file (./tmp/sales_series_melt.parquet) - preparing the data
DEBUG [2020-07-13 20:19:14] root: Read 10 series
DEBUG [2020-07-13 20:19:25] root: Setting index
DEBUG [2020-07-13 20:19:25] root: Setting index - done
DEBUG [2020-07-13 20:19:25] root: Dropping 'day_date' from Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'day_id',
       'sales', 'day_date', 'day_date_str', 'month_id', 'date', 'wm_yr_wk',
       'snap_flag', 'sell_price', 'sales_dollars'],
      dtype='object')
DEBUG [2020-07-13 20:19:25] root: Encoding: id
DEBUG [2020-07-13 20:19:26] root: Encoding: item_id
DEBUG [2020-07-13 20:19:26] root: Encoding: dept_id
DEBUG [2020-07-13 20:19:26] root: Encoding: cat_id
DEBUG [2020-07-13 20:19:27] root: Encoding: store_id
DEBUG [2020-07-13 20:19:27] root: Encoding: state_id
DEBUG [2020-07-13 20:19:27] root: Encoding: day_date_str
DEBUG [2020-07-13 20:19:28] root: Encoding: date
DEBUG [2020-07-13 20:19:30] root: 

In [ ]:
# TODO: move to core (don't forget to add import here)

from sklearn.preprocessing import LabelEncoder
import dask.dataframe as dd
def load_encoders(processed):
    def _load(fn):
        l = LabelEncoder()
        l.classes_ = np.load(f'{processed}/{fn}', allow_pickle=True)
        return l

    encoders_paths = filter(lambda p: p.endswith('.npy'), os.listdir(processed))
    encoders = {fn[:-len('.npy')]:_load(fn) for fn in encoders_paths}

    return encoders

def encode(log, me, processed):
    encoders = load_encoders(processed)
    continuous_cols = ['sell_price']

    for col in me.columns:
        dtype_str = str(me[col].dtype)
        if col in continuous_cols:
            log.debug(f"Encoding {col} ({dtype_str}) as float32 just in case for pytorch")
            me[col] = me[col].astype('float32')
            continue

        log.debug(f"Encoding {col} ({dtype_str}) as categorical ")

        unlabelable = ~me[col].isin(encoders[col].classes_)
        unlabelable_count = unlabelable.sum()
        if unlabelable_count > 0:
            default_label = encoders[col].classes_[0]
            log.warning(f"{unlabelable_count} entries for {col} can't be labeled. Defaulting to {default_label} e.g.\n {me[unlabelable][col][:3].values}")
            me.loc[unlabelable, col] = default_label

        me[col] = dd.from_array(encoders[col].transform(me[col]))

    return me

In [ ]:
raw = 'raw'
processed = './tmp'

In [ ]:
def prepare_test_data_on_disk(log, raw, processed):
    expected_path = f'{processed}/test_series_melt.parquet'
    if os.path.exists(expected_path) and not force_data_prep:
        log.info(f'Found parquet file ({expected_path})- skipping the prep')
        return

    template = get_submission_template_melt(raw)
    test_data = encode(log, template, processed)
    to_parquet(test_data, 'test_series_melt.parquet', processed, log)

In [ ]:
prepare_test_data_on_disk(log, raw='raw', processed='./tmp')

DEBUG [2020-07-13 20:37:05] root: Encoding id (object) as categorical 
WARNING [2020-07-13 20:37:06] root: 1707076 entries for id can't be labeled. Defaulting to FOODS_1_132_CA_3_validation e.g.
 ['HOBBIES_1_001_CA_1_validation' 'HOBBIES_1_002_CA_1_validation'
 'HOBBIES_1_003_CA_1_validation']
DEBUG [2020-07-13 20:37:06] root: Encoding date (object) as categorical 
WARNING [2020-07-13 20:37:07] root: 1707440 entries for date can't be labeled. Defaulting to 2011-01-29 e.g.
 [datetime.date(2016, 6, 21) datetime.date(2016, 6, 21)
 datetime.date(2016, 6, 21)]
DEBUG [2020-07-13 20:37:07] root: Encoding sell_price (float64) as float32 just in case for pytorch
DEBUG [2020-07-13 20:37:07] root: Encoding cat_id (object) as categorical 
DEBUG [2020-07-13 20:37:08] root: Encoding dept_id (object) as categorical 
WARNING [2020-07-13 20:37:08] root: 83440 entries for dept_id can't be labeled. Defaulting to FOODS_1 e.g.
 ['HOBBIES_2' 'HOBBIES_2' 'HOBBIES_2']
DEBUG [2020-07-13 20:37:09] root: Encodin

TypeError: __cinit__() got an unexpected keyword argument 'write_index'